# DNN with Hyper-Parameters Tuning

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

#hyper-parameter tuning
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}" # folder names as a timestamp

UFC_FINAL = pd.read_csv("UFC_FINAL.csv")

# encode blue and red as 1 and 0
UFC_FINAL["Winner"] = UFC_FINAL["Winner"].replace("Blue", 1)
UFC_FINAL["Winner"] = UFC_FINAL["Winner"].replace("Red", 0)

# train/test split
X = UFC_FINAL.drop("Winner", axis=1).values
y = UFC_FINAL["Winner"].values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=101)

# scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape} | X_test shape: {X_test.shape} | y_train shape: {y_train.shape} | y_test shape: {y_test.shape}")

# function to build the model (argument: hyper-parameter)
def build_model(hp):
    # model
    model = Sequential()
    
    # first layer's no. of neurons = hp.Int range of values to test
    model.add(Dense(hp.Int("input_units", min_value=30,max_value=256,step=16), activation='relu'))
    
    # range of 1 to 10 layers to test
    for i in range(hp.Int("no. Of Hidden Layers", 1, 10)):
        # for each layer, again test range of neurons
        model.add(Dense(hp.Int(f"Hidden_layer_{i+1}_units", min_value=30,max_value=256,step=16), activation='relu'))

    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

tuner = RandomSearch(
        build_model, # name of the function that builds the model
        objective="val_acc", # the thing that we're interested to trace
        max_trials = 2, # no. of combinations to try
        executions_per_trial = 1, # no. of times to train each combination (true avg)
        directory=LOG_DIR) # directory to save the outputs


X_train shape: (3476, 42) | X_test shape: (1159, 42) | y_train shape: (3476,) | y_test shape: (1159,)


In [5]:
tuner.search(x=X_train,
             y=y_train,
             epochs = 5,
#             batch_size = 64,
             validation_data=(X_test,y_test))

Train on 3476 samples, validate on 1159 samples
Epoch 1/5
3476/3476 [==============================] - ETA: 1:44 - loss: 0.6902 - acc: 0.687 - ETA: 9s - loss: 0.6699 - acc: 0.6847  - ETA: 5s - loss: 0.6489 - acc: 0.685 - ETA: 3s - loss: 0.6522 - acc: 0.672 - ETA: 2s - loss: 0.6479 - acc: 0.671 - ETA: 1s - loss: 0.6442 - acc: 0.673 - ETA: 1s - loss: 0.6342 - acc: 0.682 - ETA: 0s - loss: 0.6292 - acc: 0.686 - ETA: 0s - loss: 0.6280 - acc: 0.684 - ETA: 0s - loss: 0.6279 - acc: 0.684 - ETA: 0s - loss: 0.6298 - acc: 0.679 - ETA: 0s - loss: 0.6253 - acc: 0.682 - 2s 579us/sample - loss: 0.6253 - acc: 0.6821 - val_loss: 0.6230 - val_acc: 0.6644
Epoch 2/5
3476/3476 [==============================] - ETA: 0s - loss: 0.6648 - acc: 0.593 - ETA: 0s - loss: 0.6179 - acc: 0.687 - ETA: 0s - loss: 0.6150 - acc: 0.682 - ETA: 0s - loss: 0.5998 - acc: 0.691 - ETA: 0s - loss: 0.6067 - acc: 0.681 - ETA: 0s - loss: 0.6036 - acc: 0.685 - ETA: 0s - loss: 0.6112 - acc: 0.678 - ETA: 0s - loss: 0.6140 - acc: 0.67

Train on 3476 samples, validate on 1159 samples
Epoch 1/5
3476/3476 [==============================] - ETA: 2:35 - loss: 0.6932 - acc: 0.437 - ETA: 39s - loss: 0.6836 - acc: 0.632 - ETA: 26s - loss: 0.6809 - acc: 0.62 - ETA: 20s - loss: 0.6576 - acc: 0.67 - ETA: 14s - loss: 0.6452 - acc: 0.68 - ETA: 12s - loss: 0.6495 - acc: 0.68 - ETA: 11s - loss: 0.6488 - acc: 0.67 - ETA: 9s - loss: 0.6460 - acc: 0.6801 - ETA: 7s - loss: 0.6376 - acc: 0.686 - ETA: 5s - loss: 0.6335 - acc: 0.686 - ETA: 4s - loss: 0.6362 - acc: 0.679 - ETA: 3s - loss: 0.6366 - acc: 0.677 - ETA: 3s - loss: 0.6328 - acc: 0.683 - ETA: 2s - loss: 0.6312 - acc: 0.683 - ETA: 2s - loss: 0.6334 - acc: 0.680 - ETA: 2s - loss: 0.6347 - acc: 0.678 - ETA: 1s - loss: 0.6355 - acc: 0.676 - ETA: 1s - loss: 0.6345 - acc: 0.677 - ETA: 1s - loss: 0.6356 - acc: 0.674 - ETA: 1s - loss: 0.6324 - acc: 0.677 - ETA: 0s - loss: 0.6339 - acc: 0.675 - ETA: 0s - loss: 0.6301 - acc: 0.679 - ETA: 0s - loss: 0.6289 - acc: 0.680 - ETA: 0s - loss: 0.6

INFO:tensorflow:Oracle triggered exit


In [7]:
print(tuner.results_summary()) # top 10 trials
print(tuner.get_best_hyperparameters()[0].values) #values of best hyper-parameters
#print(tuner.get_best_models()[0].summary())

None
{'input_units': 174, 'no. Of Hidden Layers': 6, 'Hidden_layer_1_units': 126, 'Hidden_layer_2_units': 30, 'Hidden_layer_3_units': 30, 'Hidden_layer_4_units': 30, 'Hidden_layer_5_units': 30, 'Hidden_layer_6_units': 30}
